In [ ]:
import numpy as np
import scipy.integrate
import matplotlib.pyplot as plt
import random 
import copy
from itertools import compress
from itertools import groupby
import scipy.optimize as opt
import time
import pickle

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 22}

matplotlib.rc('font', **font)

In [ ]:
def system1_ODEs(t,z,params):
    k, p = z
    n, m, kba, kbb, kua, kub, gammaA, gammaB, Atot, Btot, F = params
        
    dkdt = kba * ((Atot- n * k) / (1 + gammaA * k / (p + F)))**n - kua * k
    dpdt = kbb * ((Btot- m * p) / (1 + gammaB * k / (p + F)))**m - kub * p
    
    dzdt = [dkdt, dpdt]
    return(dzdt)
    
def system1_Jac(t,z,params):
    n, m, kba, kbb, kua, kub, gammaA, gammaB, Atot, Btot, F = params
    k, p = z
    
    jac = np.array([[
            -kua + (kba*n*(n*(F + p) + Atot*gammaA)*
                    (((Atot - k*n)*(F + p))/(F + p + k*gammaA))**n)/
                    ((-Atot + k*n)*(F + p + k*gammaA)),
                    (k*kba*n*gammaA*(((Atot - k*n)*(F + p))/(F + p + k*gammaA))**n)/
                    ((F + p)*(F + p + k*gammaA))],
    [-((kbb*m*gammaB*(((F + p)*(Btot - m*p))/(F + p + k*gammaB))**m)/(F + p + k*gammaB)),
     (-(kub*(F + p + k*gammaB)**2) + kbb*m*(((F + p)*(Btot - m*p))/(F + p + k*gammaB))**(-1 + m)*
      (k*(Btot - m*p)*gammaB - m*(F + p)*(F + p + k*gammaB)))/(F + p + k*gammaB)**2]])
        
    return jac

def system1_Sol(tSpan, params, initialConditions, toPlot, plotLog=False, atol=1e-12, rtol=1e-6):
    odeSol = scipy.integrate.solve_ivp(lambda tSpan, z: system1_ODEs(tSpan, z, params),
                                        tSpan,initialConditions,method = 'Radau', vectorized=False,
                                        jac = lambda tSpan,z: system1_Jac(tSpan,z,params),
                                        atol=atol, rtol=rtol)
    z = np.transpose(odeSol.y)
    t = odeSol.t
    
    if toPlot:
        system1_Plot_for_Fig(t, z, params, initialConditions, plotLog=plotLog)
    return z,t

def system1_Plot_for_Fig(t, z, params, initialConditions, plotLog=False):
    
    n, m, kba, kbb, kua, kub, gammaA, gammaB, Atot, Btot, f = params
    k = z[:,0]
    p = z[:,1]
    lenTStart = int(len(z[:,0])/10)

    plt.plot(t[lenTStart:], k[lenTStart:], 'b', label='k')
    
    if not plotLog:
        plt.plot(t[lenTStart:], p[lenTStart:], 'k', label='p')
    else:
        plt.semilogy(t[lenTStart:], p[lenTStart:], 'k', label='p')

    plt.ylabel(r'concentration ($\mu$M)')
    plt.xlabel('time (s)')
    plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0),fontsize = 12)
    #plt.ylim(top=Atot)
    plt.show()

In [ ]:
def load(filename):
# =============================================================================
#     Load an object from a file
# =============================================================================
    if not filename[-7:] == '.pickle':
        filename = filename + '.pickle'
    with open(filename, 'rb') as handle:
        data = pickle.load(handle)
    return data

(oscParamsn2m2Exp, oscAlpha1sn2m2Exp, oscAlpha3sn2m2Exp, oscAlpha4sn2m2Exp, 
 oscPeriodsn2m2Exp, nonOscParamsn2m2Exp, nonOscAlpha1sn2m2Exp, nonOscAlpha3sn2m2Exp, 
 nonOscAlpha4sn2m2Exp) = load(
         '/Users/Ofer/Dropbox/Brenner/Baker_Oscillators/Final variables for paper/n2m2_for_fig_3.pickle')
# Each parameter set consists of:
# n, m, kba, kbb, kua, kub, gammaA, gammaB, Atot, Btot, F = params

In [ ]:
oscParamsn2m2Exp[4]

In [ ]:
#kbkio > kbkic
#kukic >= kukio
params = [2, 2, 0.81, 0.034, 0.0032, 0.024, 1.14, 1.14, 0.0010, 0.0050, 1.9e-08] # oscParamsn2m2Exp[0]
tmax = 10000 #maximum time we'll integrate to

params = [2, 2, 0.52, 0.37, 21, 96, 1.14, 1.14, 1.2, 1.2, 8.0e-08]  # oscParamsn2m2Exp[4]
tmax = 10 #maximum time we'll integrate to

n, m, kbk, kbp, kuk, kup, etaK, gammaP, ktot, ptot, F = params

initialConditions = [ktot, ptot]
tSpan = [0,tmax] #np.linspace(0,tmax,numTimePoints)

toPlot = True

z,t = system1_Sol(tSpan,params,initialConditions,toPlot,False)
k = z[:,0]
p = z[:,1]

dzdt = system1_ODEs(t[-1], z[-1,:], params)
print('max(abs(dzdt)) = ' + str(max(np.abs(dzdt))))

dpdt = (p[1:] - p[0:-1])/(t[1:]-t[0:-1])

groups = []
uniquekeys = []
counter = 0
prevCounter = 0
prevPrevCounter = 0
prevCounterArray = []
xChange = [] #how much does the value of Ao change in between each inflection point?
for k, g in groupby(np.sign(dpdt)):
    groups.append(list(g))
    c = len(uniquekeys) #counter that just increases by one in each for loop
    xChange.append(np.sum(dpdt[counter:counter+len(groups[c])]*
                          (t[counter+1:counter+len(groups[c])+1]-t[counter:counter+len(groups[c])])))
    prevPrevPrevCounter = prevPrevCounter
    prevPrevCounter = prevCounter
    prevCounter = counter
    counter += len(groups[c])
    uniquekeys.append(k)
    
    prevCounterArray += [t[counter]]
nIP = len(uniquekeys)

period = prevCounterArray[-4] - prevCounterArray[-6]
print('period = ' + str(period))
# print('kMKo = ' + str((kuko + kcko)/kbko) + '; kMKc = ' + str((kukc + kckc)/kbkc))
# print('kMPo = ' + str((kupo + kcpo)/kbpo) + '; kMPc = ' + str((kupc + kcpc)/kbpc))
# print('kcK = ' + str(kcko) + '; kcP = ' + str(kcpo))
# print('kDKo = ' + str(kukio/kbkio) + '; kDKc = ' + str(kukic/kbkic))

In [ ]:
var_list = [r'$n$', r'$m$', 
            r'$k_{b\kappa}$ $(\mu$M$^{-1}$ s$^{-1})$', r'$k_{b\rho}$ $(\mu$M$^{-1}$ s$^{-1})$', 
            r'$k_{u\kappa}$ $($s$^{-1})$', r'$k_{u\rho}$ $($s$^{-1})$', 
            r'$\eta_\kappa = \eta_\rho$', r'$\eta_\kappa = \eta_\rho$', 
            r'$\kappa_{tot}$ $(\mu$M$)$', r'$\rho_{tot}$ $(\mu$M$)$', r'$\tilde{P}$ $(\mu$M$)$']

def robustness(var_index, var_arr, logscale=False):    
    period_arr = np.zeros(len(var_arr))
    period = 0  # to initialize
    
    for e, var in enumerate(var_arr):
        params = [2, 2, 0.81, 0.034, 0.0032, 0.024, 1.14, 1.14, 0.0010, 0.0050, 1.9e-08]
        params[var_index] = var
        if var_index == 6:
            params[var_index + 1] = var
        if var_index == 7:
            params[var_index - 1] = var
            
        n, m, kbk, kbp, kuk, kup, etaK, gammaP, ktot, ptot, F = params
        
        print(var_list[var_index] + ' = ' + str(var))
        
        initialConditions = [0, 0]        
        if period == 0:
            tmax = 100000 #maximum time we'll integrate to
        else:
            tmax = period * 50
        tSpan = [0,tmax] #np.linspace(0,tmax,numTimePoints)

        toPlot = True

        z,t = system1_Sol(tSpan, params, initialConditions, toPlot, True)
        k = z[:,0]
        p = z[:,1]

        dpdt = (p[1:] - p[0:-1])/(t[1:]-t[0:-1])

        groups = []
        uniquekeys = []
        counter = 0
        prevCounter = 0
        prevPrevCounter = 0
        prevCounterArray = []
        xChange = [] #how much does the value of Ao change in between each inflection point?
        for k, g in groupby(np.sign(dpdt)):
            groups.append(list(g))
            c = len(uniquekeys) #counter that just increases by one in each for loop
            xChange.append(np.sum(dpdt[counter:counter+len(groups[c])]*
                                  (t[counter+1:counter+len(groups[c])+1]-t[counter:counter+len(groups[c])])))
            prevPrevPrevCounter = prevPrevCounter
            prevPrevCounter = prevCounter
            prevCounter = counter
            counter += len(groups[c])
            uniquekeys.append(k)

            prevCounterArray += [t[counter]]
        nIP = len(uniquekeys)

        if (nIP > 10 and min(np.abs(xChange[:-1] / xChange[5])) > 0.5 and 
            t[-3]/t[-2] > 0.95 and abs((abs(xChange[3*len(xChange)//4]) - abs(xChange[-2]))/xChange[-2]) < 1 and 
            not (z<-10000).any()):
            period = prevCounterArray[-4] - prevCounterArray[-6]
        else:
            period = 0
        print('period = ' + str(period))
        print('nIP = ' + str(nIP))
        period_arr[e] = period
        
    plt.figure()
    plt.plot(var_arr, period_arr, '.')
    plt.xlabel(var_list[var_index])
    plt.ylabel(r'Period $(s)$')
    if logscale:
        plt.xscale('log')
    plt.show()
    
    return(period_arr)

In [ ]:
logscale = True
if logscale:
    kbks = np.logspace(-1,1,50, base=10)
else:
    kbks = np.linspace(1e-1, 4, 50)
period_fxn_kbk = robustness(2, kbks, logscale)

In [ ]:
logscale = True
if logscale:
    kbps = np.logspace(-2, -0.6,50, base=10)
else:
    kbps = np.linspace(1e-2,2e-1,50)

period_fxn_kbp = robustness(3, kbps, logscale)

In [ ]:
logscale = True
if logscale:
    kuks = np.logspace(-4,-2,50, base=10)
else:
    kuks = np.linspace(1e-4, 7e-3, 50)

period_fxn_kuk = robustness(4, kuks, logscale)

In [ ]:
logscale = True
if logscale:
    kups = np.logspace(-2,-0.8,50, base=10)
else:
    kups = np.linspace(1e-2, 1e-1, 50)

period_fxn_kup = robustness(5, kups, logscale)

In [ ]:
# period_fxn_ktot = robustness(8, np.logspace(-4,-2,20, base=10), True)
ktots = np.linspace(1e-4, 2.5e-3, 50)
period_fxn_ktot = robustness(8, ktots, False)

In [ ]:
# period_fxn_ptot = robustness(9, np.logspace(-3,-1,20, base=10), True)
ptots = np.linspace(1e-3, 1.5e-2, 50)
period_fxn_ptot = robustness(9, ptots, False)

In [ ]:
ptildes = np.logspace(-12, -6, 50, base=10)
period_fxn_ptilde = robustness(10, ptildes, True)

In [ ]:
etas = np.logspace(-1, 1, 50, base=10)
period_fxn_eta = robustness(6, etas, True)

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(15,7))

axs[0, 0].plot(kbks[period_fxn_kbk > 0], period_fxn_kbk[period_fxn_kbk>0])
axs[0, 0].plot(kbks[period_fxn_kbk == 0], period_fxn_kbk[period_fxn_kbk == 0], '.')
axs[0, 0].set_xlabel(var_list[2])
axs[0, 0].set_xscale('log')
axs[0, 0].set_xticks([1e-1, 1e0, 1e1])
# axs[0, 0].xaxis.grid(True, which='minor')
# axs[0, 0].tick_params(axis='x', which='minor')
# axs[0, 0].get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
axs[0, 0].set_ylabel('Period (s)')
axs[0, 0].set_ylim([-200, 5000])

axs[0, 1].plot(kbps[period_fxn_kbp > 0], period_fxn_kbp[period_fxn_kbp > 0])
axs[0, 1].plot(kbps[period_fxn_kbp == 0], period_fxn_kbp[period_fxn_kbp == 0], '.')
axs[0, 1].set_xlabel(var_list[3])
axs[0, 1].set_xscale('log')
axs[0, 1].set_xticks([1e-2, 1e-1])
axs[0, 1].set_ylim([-200, 5000])
axs[0, 1].set_yticks([])

axs[0, 2].plot(kuks[period_fxn_kuk>0], period_fxn_kuk[period_fxn_kuk>0])
axs[0, 2].plot(kuks[period_fxn_kuk ==0], period_fxn_kuk[period_fxn_kuk==0], '.')
axs[0, 2].set_xlabel(var_list[4])
axs[0, 2].set_xscale('log')
axs[0, 2].set_xticks([1e-4, 1e-3, 1e-2])
axs[0, 2].set_ylim([-200, 5000])
axs[0, 2].set_yticks([])

axs[0, 3].plot(kups[period_fxn_kup>0], period_fxn_kup[period_fxn_kup>0])
axs[0, 3].plot(kups[period_fxn_kup==0], period_fxn_kup[period_fxn_kup==0], '.')
axs[0, 3].set_xlabel(var_list[5])
axs[0, 3].set_xscale('log')
axs[0, 3].set_xticks([1e-2, 1e-1])
axs[0, 3].set_ylim([-200, 5000])
axs[0, 3].set_yticks([])

axs[1, 0].plot(ktots[period_fxn_ktot>0], period_fxn_ktot[period_fxn_ktot>0])
axs[1, 0].plot(ktots[period_fxn_ktot==0], period_fxn_ktot[period_fxn_ktot==0], '.')
axs[1, 0].set_xlabel(r'$\kappa_{tot}$ (nM)') #var_list[8])
axs[1, 0].set_ylabel('Period (s)')
axs[1, 0].set_xticks([0, 0.001, 0.002])
axs[1, 0].set_xticklabels(['0', '1', '2'])
# plt.ticklabel_format(style='sci', axis='x', scilimits=(-5,-5))
axs[1, 0].set_ylim([-200, 5000])

axs[1, 1].plot(ptots[period_fxn_ptot>0], period_fxn_ptot[period_fxn_ptot>0])
axs[1, 1].plot(ptots[period_fxn_ptot==0], period_fxn_ptot[period_fxn_ptot==0], '.')
axs[1, 1].set_xlabel(r'$\rho_{tot}$ (nM)') #var_list[9])
axs[1, 1].set_xticks([0, 5e-3, 1e-2, 1.5e-2])
axs[1, 1].set_xticklabels(['0', '5', '10', '15'])
axs[1, 1].set_ylim([-200, 5000])
axs[1, 1].set_yticks([])

axs[1, 2].plot(ptildes[period_fxn_ptilde>0], period_fxn_ptilde[period_fxn_ptilde>0])
axs[1, 2].plot(ptildes[period_fxn_ptilde==0], period_fxn_ptilde[period_fxn_ptilde==0], '.')
axs[1, 2].set_xlabel(var_list[10])
axs[1, 2].set_xscale('log')
axs[1, 2].set_xticks([1e-12, 1e-9, 1e-6])
axs[1, 2].set_ylim([-200, 5000])
axs[1, 2].set_yticks([])

axs[1, 3].plot(etas[period_fxn_eta>0], period_fxn_eta[period_fxn_eta>0])
axs[1, 3].plot(etas[period_fxn_eta==0], period_fxn_eta[period_fxn_eta==0],'.')
axs[1, 3].set_xlabel(var_list[6])
axs[1, 3].set_xscale('log')
axs[1, 3].set_xticks([1e-1, 1e0, 1e1])
axs[1, 3].set_ylim([-200, 5000])
axs[1, 3].set_yticks([])

plt.tight_layout(h_pad=0, w_pad=0.2)
# plt.savefig('robustness_bounded.png', 
#             dpi=800, bbox_inches='tight')
# plt.savefig('robustness_bounded_small.png', 
#             bbox_inches='tight')
plt.show()

In [ ]:
var_list = [r'$n$', r'$m$', 
            r'$k_{b\kappa}$ $(\mu$M$^{-1}$ s$^{-1})$', r'$k_{b\rho}$ $(\mu$M$^{-1}$ s$^{-1})$', 
            r'$k_{u\kappa}$ $($s$^{-1})$', r'$k_{u\rho}$ $($s$^{-1})$', 
            r'$\eta_\kappa = \eta_\rho$', r'$\eta_\kappa = \eta_\rho$', 
            r'$\kappa_{tot}$ $(\mu$M$)$', r'$\rho_{tot}$ $(\mu$M$)$', r'$\tilde{P}$ $(\mu$M$)$']

def robustness2(var_index, var_arr, logscale=False, tmax_init=100):    
    period_arr = np.zeros(len(var_arr))
    period = 0  # to initialize
    
    for e, var in enumerate(var_arr):
        params = [2, 2, 0.52, 0.37, 21, 96, 1.14, 1.14, 1.2, 1.2, 8.0e-08]  # oscParamsn2m2Exp[4]
        params[var_index] = var
        if var_index == 6:
            params[var_index + 1] = var
        if var_index == 7:
            params[var_index - 1] = var
            
        n, m, kbk, kbp, kuk, kup, etaK, gammaP, ktot, ptot, F = params
        
        print(var_list[var_index] + ' = ' + str(var))
        
        initialConditions = [0, 0]        
        if period == 0:
            tmax = tmax_init #maximum time we'll integrate to
        else:
            tmax = period * 50
        tSpan = [0,tmax] #np.linspace(0,tmax,numTimePoints)

        toPlot = True

        z,t = system1_Sol(tSpan, params, initialConditions, toPlot, True)
        k = z[:,0]
        p = z[:,1]

        dpdt = (p[1:] - p[0:-1])/(t[1:]-t[0:-1])

        groups = []
        uniquekeys = []
        counter = 0
        prevCounter = 0
        prevPrevCounter = 0
        prevCounterArray = []
        xChange = [] #how much does the value of Ao change in between each inflection point?
        for k, g in groupby(np.sign(dpdt)):
            groups.append(list(g))
            c = len(uniquekeys) #counter that just increases by one in each for loop
            xChange.append(np.sum(dpdt[counter:counter+len(groups[c])]*
                                  (t[counter+1:counter+len(groups[c])+1]-t[counter:counter+len(groups[c])])))
            prevPrevPrevCounter = prevPrevCounter
            prevPrevCounter = prevCounter
            prevCounter = counter
            counter += len(groups[c])
            uniquekeys.append(k)

            prevCounterArray += [t[counter]]
        nIP = len(uniquekeys)

        if (nIP > 10 and min(np.abs(xChange[:-1] / xChange[5])) > 0.5 and 
            t[-3]/t[-2] > 0.95 and abs((abs(xChange[3*len(xChange)//4]) - abs(xChange[-2]))/xChange[-2]) < 1 and 
            not (z<-10000).any()):
            period = prevCounterArray[-4] - prevCounterArray[-6]
        else:
            period = 0
        print('period = ' + str(period))
        print('nIP = ' + str(nIP))
        period_arr[e] = period
        
    plt.figure()
    plt.plot(var_arr, period_arr, '.')
    plt.xlabel(var_list[var_index])
    plt.ylabel(r'Period $(s)$')
    if logscale:
        plt.xscale('log')
    plt.show()
    
    return(period_arr)

In [ ]:
num_points = 50

In [ ]:
logscale = True
if logscale:
    kbks2 = np.logspace(-1,1,num_points, base=10)
else:
    kbks2 = np.linspace(1e-1, 4, num_points)
period_fxn_kbk2 = robustness2(2, kbks2, logscale)

In [ ]:
logscale = True
if logscale:
    kbps2 = np.logspace(-2, 0.3, num_points, base=10)
else:
    kbps2 = np.linspace(1e-2, 2e0, num_points)

period_fxn_kbp2 = robustness2(3, kbps2, logscale)

In [ ]:
logscale = True
if logscale:
    kuks2 = np.logspace(-0.8, 1.7, num_points, base=10)
else:
    kuks2 = np.linspace(0.2, 5e1, num_points)

period_fxn_kuk2 = robustness2(4, kuks2, logscale, tmax_init=1e4)

In [ ]:
logscale = True
if logscale:
    kups2 = np.logspace(1.7, 4, num_points, base=10)
else:
    kups2 = np.linspace(5e1, 1e4, num_points)

period_fxn_kup2 = robustness2(5, kups2, logscale)

In [ ]:
# period_fxn_ktot = robustness(8, np.logspace(-4,-2,20, base=10), True)
ktots2 = np.linspace(1e-1, 7.5, num_points)
period_fxn_ktot2 = robustness2(8, ktots2, False)

In [ ]:
# period_fxn_ptot = robustness(9, np.logspace(-3,-1,20, base=10), True)
ptots2 = np.linspace(1e-1, 2.5, num_points)
period_fxn_ptot2 = robustness2(9, ptots2, False)

In [ ]:
ptildes2 = np.logspace(-12, -4, num_points, base=10)
period_fxn_ptilde2 = robustness2(10, ptildes2, True)

In [ ]:
etas2 = np.logspace(-1, 2, num_points, base=10)
period_fxn_eta2 = robustness2(6, etas2, True)

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(15,7))

axs[0, 0].plot(kbks2[period_fxn_kbk2 > 0], period_fxn_kbk2[period_fxn_kbk2>0])
axs[0, 0].plot(kbks2[period_fxn_kbk2 == 0], period_fxn_kbk2[period_fxn_kbk2 == 0], '.')
axs[0, 0].set_xlabel(var_list[2])
axs[0, 0].set_xscale('log')
axs[0, 0].set_xticks([1e-1, 1e0, 1e1])
# axs[0, 0].xaxis.grid(True, which='minor')
# axs[0, 0].tick_params(axis='x', which='minor')
# axs[0, 0].get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
axs[0, 0].set_ylabel('Period (s)')
axs[0, 0].set_yscale('symlog', linthreshy=0.2)
axs[0, 0].set_ylim([-0.1, 15])
axs[0, 0].set_yticks([0, 1e-1, 1e0, 1e1])

axs[0, 1].plot(kbps2[period_fxn_kbp2 > 0], period_fxn_kbp2[period_fxn_kbp2 > 0])
axs[0, 1].plot(kbps2[period_fxn_kbp2 == 0], period_fxn_kbp2[period_fxn_kbp2 == 0], '.')
axs[0, 1].set_xlabel(var_list[3])
axs[0, 1].set_xscale('log')
axs[0, 1].set_xticks([1e-2, 1e-1, 1e0])
axs[0, 1].set_ylim([-0.1, 15])
axs[0, 1].set_yscale('symlog', linthreshy=0.2)
axs[0, 1].set_yticks([])

axs[0, 2].plot(kuks2[period_fxn_kuk2>0], period_fxn_kuk2[period_fxn_kuk2>0])
axs[0, 2].plot(kuks2[period_fxn_kuk2 ==0], period_fxn_kuk2[period_fxn_kuk2==0], '.')
axs[0, 2].set_xlabel(var_list[4])
axs[0, 2].set_xscale('log')
axs[0, 2].set_xticks([1e0, 1e1])
axs[0, 2].set_ylim([-0.1, 15])
axs[0, 2].set_yscale('symlog', linthreshy=0.2)
axs[0, 2].set_yticks([])

axs[0, 3].plot(kups2[period_fxn_kup2>0], period_fxn_kup2[period_fxn_kup2>0])
axs[0, 3].plot(kups2[period_fxn_kup2==0], period_fxn_kup2[period_fxn_kup2==0], '.')
axs[0, 3].set_xlabel(var_list[5])
axs[0, 3].set_xscale('log')
axs[0, 3].set_xticks([1e2, 1e3, 1e4])
axs[0, 3].set_ylim([-0.1, 15])
axs[0, 3].set_yscale('symlog', linthreshy=0.2)
axs[0, 3].set_yticks([])

axs[1, 0].plot(ktots2[period_fxn_ktot2>0], period_fxn_ktot2[period_fxn_ktot2>0])
axs[1, 0].plot(ktots2[period_fxn_ktot2==0], period_fxn_ktot2[period_fxn_ktot2==0], '.')
axs[1, 0].set_xlabel(var_list[8])
axs[1, 0].set_ylabel('Period (s)')
axs[1, 0].set_xticks([0, 2, 4, 6, 8])
# axs[1, 0].set_xticklabels(['0', r'$10^{-3}$', r'$2\times 10^{-3}$'])
# plt.ticklabel_format(style='sci', axis='x', scilimits=(-5,-5))
axs[1, 0].set_ylim([-0.1, 15])
axs[1, 0].set_yscale('symlog', linthreshy=0.2)
axs[1, 0].set_yticks([0, 1e-1, 1e0, 1e1])

axs[1, 1].plot(ptots2[period_fxn_ptot2>0], period_fxn_ptot2[period_fxn_ptot2>0])
axs[1, 1].plot(ptots2[period_fxn_ptot2==0], period_fxn_ptot2[period_fxn_ptot2==0], '.')
axs[1, 1].set_xlabel(var_list[9])
axs[1, 1].set_xticks([0, 1, 2])
# axs[1, 1].set_xticklabels(['0', '5e-3', '1e-2', '1.5e-2'])
axs[1, 1].set_ylim([-0.1, 15])
axs[1, 1].set_yscale('symlog', linthreshy=0.2)
axs[1, 1].set_yticks([])

axs[1, 2].plot(ptildes2[period_fxn_ptilde2>0], period_fxn_ptilde2[period_fxn_ptilde2>0])
axs[1, 2].plot(ptildes2[period_fxn_ptilde2==0], period_fxn_ptilde2[period_fxn_ptilde2==0], '.')
axs[1, 2].set_xlabel(var_list[10])
axs[1, 2].set_xscale('log')
axs[1, 2].set_xticks([1e-12, 1e-8, 1e-4])
axs[1, 2].set_ylim([-0.1, 15])
axs[1, 2].set_yscale('symlog', linthreshy=0.2)
axs[1, 2].set_yticks([])

axs[1, 3].plot(etas2[period_fxn_eta2>0], period_fxn_eta2[period_fxn_eta2>0])
axs[1, 3].plot(etas2[period_fxn_eta2==0], period_fxn_eta2[period_fxn_eta2==0],'.')
axs[1, 3].set_xlabel(var_list[6])
axs[1, 3].set_xscale('log')
axs[1, 3].set_xticks([1e-1, 1e0, 1e1, 1e2])
axs[1, 3].set_ylim([-0.1, 15])
axs[1, 3].set_yscale('symlog', linthreshy=0.2)
axs[1, 3].set_yticks([])

plt.tight_layout(h_pad=0, w_pad=0.2)
plt.savefig('/Users/Ofer/Dropbox/Brenner/Baker_Oscillators/Final figures for paper/robustness_bounded_2.png', 
            dpi=800, bbox_inches='tight')
plt.savefig('/Users/Ofer/Dropbox/Brenner/Baker_Oscillators/Final figures for paper/robustness_bounded_2_small.png', 
            bbox_inches='tight')
plt.show()

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(15,7))

axs[0, 0].plot(kbks2[period_fxn_kbk2 > 0], period_fxn_kbk2[period_fxn_kbk2>0])
axs[0, 0].plot(kbks2[period_fxn_kbk2 == 0], period_fxn_kbk2[period_fxn_kbk2 == 0], '.')
axs[0, 0].set_xlabel(var_list[2])
axs[0, 0].set_xscale('log')
axs[0, 0].set_xticks([1e-1, 1e0, 1e1])
# axs[0, 0].xaxis.grid(True, which='minor')
# axs[0, 0].tick_params(axis='x', which='minor')
# axs[0, 0].get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
axs[0, 0].set_ylabel('Period (s)')
# axs[0, 0].set_yscale('symlog', linthreshy=0.5)
axs[0, 0].set_ylim([-0.4, 10])

axs[0, 1].plot(kbps2[period_fxn_kbp2 > 0], period_fxn_kbp2[period_fxn_kbp2 > 0])
axs[0, 1].plot(kbps2[period_fxn_kbp2 == 0], period_fxn_kbp2[period_fxn_kbp2 == 0], '.')
axs[0, 1].set_xlabel(var_list[3])
axs[0, 1].set_xscale('log')
axs[0, 1].set_xticks([1e-2, 1e-1, 1e0])
axs[0, 1].set_ylim([-0.4, 10])
# axs[0, 1].set_yscale('symlog', linthreshy=0.5)
axs[0, 1].set_yticks([])

axs[0, 2].plot(kuks2[period_fxn_kuk2>0], period_fxn_kuk2[period_fxn_kuk2>0])
axs[0, 2].plot(kuks2[period_fxn_kuk2 ==0], period_fxn_kuk2[period_fxn_kuk2==0], '.')
axs[0, 2].set_xlabel(var_list[4])
axs[0, 2].set_xscale('log')
axs[0, 2].set_xticks([1e0, 1e1])
axs[0, 2].set_ylim([-0.4, 10])
# axs[0, 2].set_yscale('symlog', linthreshy=0.5)
axs[0, 2].set_yticks([])

axs[0, 3].plot(kups2[period_fxn_kup2>0], period_fxn_kup2[period_fxn_kup2>0])
axs[0, 3].plot(kups2[period_fxn_kup2==0], period_fxn_kup2[period_fxn_kup2==0], '.')
axs[0, 3].set_xlabel(var_list[5])
axs[0, 3].set_xscale('log')
axs[0, 3].set_xticks([1e2, 1e3, 1e4])
axs[0, 3].set_ylim([-0.4, 10])
# axs[0, 3].set_yscale('symlog', linthreshy=0.5)
axs[0, 3].set_yticks([])

axs[1, 0].plot(ktots2[period_fxn_ktot2>0], period_fxn_ktot2[period_fxn_ktot2>0])
axs[1, 0].plot(ktots2[period_fxn_ktot2==0], period_fxn_ktot2[period_fxn_ktot2==0], '.')
axs[1, 0].set_xlabel(var_list[8])
axs[1, 0].set_ylabel('Period (s)')
axs[1, 0].set_xticks([0, 2, 4, 6, 8])
# axs[1, 0].set_xticklabels(['0', r'$10^{-3}$', r'$2\times 10^{-3}$'])
# plt.ticklabel_format(style='sci', axis='x', scilimits=(-5,-5))
axs[1, 0].set_ylim([-0.4, 10])
# axs[1, 0].set_yscale('symlog', linthreshy=0.5)
# axs[1, 0].set_yticks([0, 1e-1, 1e0, 1e1])

axs[1, 1].plot(ptots2[period_fxn_ptot2>0], period_fxn_ptot2[period_fxn_ptot2>0])
axs[1, 1].plot(ptots2[period_fxn_ptot2==0], period_fxn_ptot2[period_fxn_ptot2==0], '.')
axs[1, 1].set_xlabel(var_list[9])
axs[1, 1].set_xticks([0, 1, 2])
# axs[1, 1].set_xticklabels(['0', '5e-3', '1e-2', '1.5e-2'])
axs[1, 1].set_ylim([-0.4, 10])
# axs[1, 1].set_yscale('symlog', linthreshy=0.5)
axs[1, 1].set_yticks([])

axs[1, 2].plot(ptildes2[period_fxn_ptilde2>0], period_fxn_ptilde2[period_fxn_ptilde2>0])
axs[1, 2].plot(ptildes2[period_fxn_ptilde2==0], period_fxn_ptilde2[period_fxn_ptilde2==0], '.')
axs[1, 2].set_xlabel(var_list[10])
axs[1, 2].set_xscale('log')
axs[1, 2].set_xticks([1e-12, 1e-8, 1e-4])
axs[1, 2].set_ylim([-0.4, 10])
# axs[1, 2].set_yscale('symlog', linthreshy=0.5)
axs[1, 2].set_yticks([])

axs[1, 3].plot(etas2[period_fxn_eta2>0], period_fxn_eta2[period_fxn_eta2>0])
axs[1, 3].plot(etas2[period_fxn_eta2==0], period_fxn_eta2[period_fxn_eta2==0],'.')
axs[1, 3].set_xlabel(var_list[6])
axs[1, 3].set_xscale('log')
axs[1, 3].set_xticks([1e-1, 1e0, 1e1, 1e2])
axs[1, 3].set_ylim([-0.4, 10])
# axs[1, 3].set_yscale('symlog', linthreshy=0.5)
axs[1, 3].set_yticks([])

plt.tight_layout(h_pad=0, w_pad=0.2)

plt.show()